## Real Estate Dataset - Data Cleaning and Preprocessing 

 In this notebook, we focus on **data preparation**, **cleaning**, and **preprocessing** for the **Real Estate Dataset**, a popular dataset often used for classification tasks predicting how real estates are sold over a peroid of time here we only use three years from 2016 to 2019  based on asessed value,sales ratio and sales amount attributes.

Good data preprocessing is crucial for reliable and interpretable results in machine learning and analytics workflows. Here, we address common data issues such as **missing values, duplicates,outliers and inconsistent categorical labels** while creating derived features to improve downstream analysis.


We start by importing essential Python libraries for data handling and manipulation.

- `pandas` for structured data operations.

- `numpy` for numerical operations.

- `os` for interacting with the operating system and directory structures.

In [1]:
import pandas as pd 
import numpy as np 
import os 

## Define and Create Paths

## Define and Create Paths
To ensure reproducibility andorganized storage, we programmatically create directories for:

- **raw data**
- **processed data**
- **results**
- **documentation**

These directories will store intermediate and final outputs for reproducibility.


In [2]:
# Get working directory 
current_dir = os.getcwd()

# Go one directory up to the root directory 
project_root_dir = os.path.dirname(current_dir)

data_dir = os.path.join(project_root_dir, 'data')
raw_dir = os.path.join(data_dir,'raw')
processed_dir = os.path.join(data_dir,'processed')

# Define paths to results folder 
results_dir = os.path.join(project_root_dir,'results')

# Define paths to docs folder 
docs_dir = os.path.join(project_root_dir,'docs') 

#Create directories if they do not exist 
os.makedirs(raw_dir,exist_ok= True)
os.makedirs(processed_dir,exist_ok= True)
os.makedirs(results_dir,exist_ok= True)
os.makedirs(data_dir,exist_ok= True)

 ## Read in the data

 ## Read in the data
 We load the **Real Estate dataset** as a CSV file.

Key considerations here are:

- **Outlier Removal:** To improve the reliability of our analysis, we identified and removed outliers using the Interquartile Range (IQR) method. This was applied to key numerical columns such as **Sale Amount, Sales Ratio, and Assessed Value** to eliminate extreme values that could distort the results.
- We use `skipinitialspace = True` to remove extra spaces after delimeters which is common in text-based datasets.

After loading, we inspect the first few rows.


In [5]:
real_estate_filename = os.path.join(raw_dir, 'Real_Estate_Sales_2016-2019_GL.csv')
real_estate = pd.read_csv(real_estate_filename,low_memory=False, skipinitialspace= True)
# Reading dataset with its header
real_estate.head(10)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,160091,2016,12/23/2016,Avon,2 EDGEWOOD,143390,224000.0,0.640134,Condo,Condo
1,160172,2016,4/13/2017,Bethel,66 H NASHVILLE ROAD,80500,130000.0,0.619231,Condo,Condo
2,160258,2016,6/29/2017,Bethel,1 EAGLE ROCK HILL,117180,200000.0,0.585900,Condo,Condo
3,16233,2016,6/9/2017,Darien,54 KENSETT LANE,887600,1495000.0,0.593712,Condo,Condo
4,1600249,2016,5/12/2017,Guilford,66-10 HIGH ST,409250,665000.0,0.615414,Condo,Condo
5,160103,2016,11/30/2016,Branford,137 PEDDLARS DR,106700,155000.0,0.688387,Condo,Condo
6,167930,2016,9/27/2017,Bridgeport,95 LANCE CIR,105820,148000.0,0.715000,Condo,Condo
7,16533,2016,8/3/2017,Newington,239 STERLING DR,238150,331100.0,0.719269,Condo,Condo
8,16005,2016,10/11/2016,Durham,83 STAGECOACH RD,114590,160000.0,0.716187,Condo,Condo
9,160239,2016,6/9/2017,New Canaan,29 MAPLE ST #3,388640,436000.0,0.891376,Condo,Condo


We also inspect the dataset's shape. We see that the data has *183,278* rows and *10* columns, before removing outliers and it now has *143,967* rows and *10* columns after removing outliers

In [16]:
real_estate.shape

(143967, 10)

In addition, we check the data types using `.info`

In [17]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143967 entries, 0 to 183275
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Serial Number     143967 non-null  int64  
 1   List Year         143967 non-null  int64  
 2   Date Recorded     143967 non-null  object 
 3   Town              143967 non-null  object 
 4   Address           143964 non-null  object 
 5   Assessed Value    143967 non-null  int64  
 6   Sale Amount       143967 non-null  float64
 7   Sales Ratio       143967 non-null  float64
 8   Property Type     143967 non-null  object 
 9   Residential Type  143967 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 12.1+ MB


In [18]:
real_estate.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type'],
      dtype='object')

By removing the outliers

In [9]:
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


In [10]:

real_estate = remove_outliers_iqr(real_estate, 'Sale Amount')
real_estate = remove_outliers_iqr(real_estate, 'Sales Ratio')
real_estate = remove_outliers_iqr(real_estate, 'Assessed Value')


Finally we check for the total rows after removing the outliers

In [19]:
print("Original:", len(real_estate))
print("After removing Sale Amount outliers:", len(remove_outliers_iqr(real_estate, 'Sale Amount')))


Original: 143967
After removing Sale Amount outliers: 141874


## Understanding the Dataset

Before proceeding with the cleaning, we would like to understanding the variables deeply. This would help guide the cleaning process. The subsequent tables detail the types, meaning and values or ranges of the variables in the dataset.


After removing the outliers we now check for unique values in each column

In [20]:
for col in real_estate.columns:
    uniques = np.unique(real_estate[col].to_list())
    print(f"Unique values in '{col}': {uniques}\n")


Unique values in 'Serial Number': [      161       162       163 ... 190700025 190700026 192000007]

Unique values in 'List Year': [2016 2017 2018 2019]

Unique values in 'Date Recorded': ['1/1/2018' '1/1/2019' '1/10/2017' ... '9/9/2018' '9/9/2019' '9/9/2020']

Unique values in 'Town': ['Andover' 'Ansonia' 'Ashford' 'Avon' 'Barkhamsted' 'Beacon Falls'
 'Berlin' 'Bethany' 'Bethel' 'Bethlehem' 'Bloomfield' 'Bolton' 'Bozrah'
 'Branford' 'Bridgeport' 'Bridgewater' 'Bristol' 'Brookfield' 'Brooklyn'
 'Burlington' 'Canaan' 'Canterbury' 'Canton' 'Chaplin' 'Cheshire'
 'Chester' 'Clinton' 'Colchester' 'Colebrook' 'Columbia' 'Cornwall'
 'Coventry' 'Cromwell' 'Danbury' 'Darien' 'Deep River' 'Derby' 'Durham'
 'East Granby' 'East Haddam' 'East Hampton' 'East Hartford' 'East Haven'
 'East Lyme' 'East Windsor' 'Eastford' 'Easton' 'Ellington' 'Enfield'
 'Essex' 'Fairfield' 'Farmington' 'Franklin' 'Glastonbury' 'Goshen'
 'Granby' 'Greenwich' 'Griswold' 'Groton' 'Guilford' 'Haddam' 'Hamden'
 'Hampton' 'H


### **Table 1: Summary of Numeric Variables**
| Variable         | Description                                   | Values / Range (excluding NaN)           |
|------------------|-----------------------------------------------|------------------------------------------|
| Serial Number    | Unique identifier for each property listing   | From 161 to 192000007                    |
| List Year        | Year the property was listed                  | 2016 – 2019                              |
| Assessed Value   | Government-estimated property value           | 1,680 – 380,370                          |
| Sale Amount      | Final sale price of the property              | 2,000 – 684,000 (after removing outliers)|
| Sales Ratio      | Sale Amount divided by Assessed Value         | ~0.29 – 1.07                         
### **Table 2: Categorical Variables Table**
| Variable             | Description                                              | Unique Values or Notes                         |
|----------------------|----------------------------------------------------------|------------------------------------------------|
| Date Recorded        | Date the transaction was officially recorded             | Dates from 2016 to 2020                        |
| Town                 | Town or city where the property is located               | 169 unique towns (e.g., Hartford)              |
| Address              | Street address of the property                           | Various values (some missing or 'nan')         |
| Property Type        | General classification of the property                   | Residential                                    |
| Residential Category | Sub-type of residential property structure               | Condo, Single Family, Two Family, etc.   |        |
    |



This dataset contains records of residential property sales, including details such as location, assessed value, sale amount, and property type. The data can be used to analyze property values, sale prices, and sales ratios across different towns and residential property types.
- Serial Number: represents  the unique number for each recorded property
- List Year: Represents the period of recording as we know it is for 3 years 2016-2019
- Date Recorded: this represents the actual date that a transaction took place
- Town: represents a specific towm in USA
- Address: uniquely identifies property location
- Assessed Value : The value estimated for tax or official purposes
- Sale Amount: The actual price the property was sold for
- Sales Ratio: The ratio of assessed value to sale amount
- Property Type:  Indicates the property is residential
-  Residential Type: The type of residential property (e.g. single-family, condo) 

### 2. Dealing with Missing Values

Using `.isnull().sum()`, we identify columns with missing values. They are:

- `address` with 3 missing values
We address these by:

- Imputing categorical missing values with `Unknown` for the columns `address`

This has been done to preserve data consistency while acknowledging uncertainity.
We inspect one more time to ensure we don't have any missing values.


In [27]:
real_estate.isnull().sum()

Serial Number       0
List Year           0
Date Recorded       0
Town                0
Address             3
Assessed Value      0
Sale Amount         0
Sales Ratio         0
Property Type       0
Residential Type    0
dtype: int64

In [29]:
real_estate['Address'] = real_estate['Address'].fillna('unknown')

In [30]:
real_estate.isnull().sum()

Serial Number       0
List Year           0
Date Recorded       0
Town                0
Address             0
Assessed Value      0
Sale Amount         0
Sales Ratio         0
Property Type       0
Residential Type    0
dtype: int64

### 3. Deal with Duplicates
Duplicates can distort statistical summaries and model performance. Using `.duplicated().sum()`, we count duplicate records.


We then inspect the duplicated records.


In [31]:
real_estate.duplicated().sum()

0

### 4. Standardize Categorical Variables
**Remove any leading or trailing spaces and convert the strings to lowercase**

To prepare categorical variables for consistent processing, we first of all remove extra spaces and convert them to lowercase. This step ensures categorical variables are clean and consistently organized.


In [33]:
real_estate['Residential Type'].unique()

array(['Condo', 'Single Family', 'Two Family', 'Three Family',
       'Four Family'], dtype=object)

In [34]:
real_estate['Residential Type'] = real_estate['Residential Type'].replace({
    'Condo': 'appartments',
    'Single Family': 'single-residence',
    'Two Family': 'duplex-residence',
    'Three Family': 'triplex-residence',
    'Four Family': 'quadplex-residence'
     
})

In [36]:
real_estate['Residential Type'].unique()


array(['appartments', 'single-residence', 'duplex-residence',
       'triplex-residence', 'quadplex-residence'], dtype=object)

In [37]:
real_estate

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type
0,160091,2016,12/23/2016,Avon,2 EDGEWOOD,143390,224000.0,0.640134,Condo,appartments
1,160172,2016,4/13/2017,Bethel,66 H NASHVILLE ROAD,80500,130000.0,0.619231,Condo,appartments
2,160258,2016,6/29/2017,Bethel,1 EAGLE ROCK HILL,117180,200000.0,0.585900,Condo,appartments
5,160103,2016,11/30/2016,Branford,137 PEDDLARS DR,106700,155000.0,0.688387,Condo,appartments
6,167930,2016,9/27/2017,Bridgeport,95 LANCE CIR,105820,148000.0,0.715000,Condo,appartments
...,...,...,...,...,...,...,...,...,...,...
183270,190120,2019,4/28/2020,Rocky Hill,174 VALLEY CREST DRIVE,174650,242000.0,0.721700,Single Family,single-residence
183271,190460,2019,8/17/2020,Southbury,10 CONCORD CT,300250,412000.0,0.728800,Condo,appartments
183273,19200,2019,6/29/2020,Monroe,39 FARM VIEW RD,238400,377400.0,0.631700,Single Family,single-residence
183274,190480,2019,2/25/2020,New Haven,498 HOWARD AV,110740,248000.0,0.446500,Two Family,duplex-residence


In [38]:
final_file  = os.path.join(processed_dir,'real_estate_cleaned.csv')
real_estate.to_csv(final_file, index=False)